In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
from dotenv import load_dotenv
import psycopg2
folder_path = 'excel_data'
# folder_path = 'excel files'


In [26]:
dataframes_dict = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)    
        df = pd.read_excel(file_path,None,header=7) 
        dataframes_dict[filename] = df
        print(dataframes_dict[filename].keys()) 
        


dict_keys(['30 March 2024'])


In [27]:
# dataframes_dict['Book_Chapter_Publication.xlsx']['2022-23']
# dataframes_dict['Book_Chapter_Publication.xlsx']['2023-24']
# dataframes_dict['Conference_Publication.xlsx']['2022-23']
# dataframes_dict['Conference_Publication.xlsx']['2023-24']
# dataframes_dict['Journal_Papers.xlsx']['2022-23']
# dataframes_dict['Journal_Papers.xlsx']['Summary of 2022-23']

In [28]:
for key, value in dataframes_dict.items():
    print("File: ",key)
    for k,v in value.items():
        print("\tSheet: ",k)
        

File:  30 March 2024.xlsx
	Sheet:  30 March 2024


In [29]:
del dataframes_dict['Journal_Papers.xlsx']['Summary of 2022-23']

KeyError: 'Journal_Papers.xlsx'

In [30]:
dataframes_dict['Book_Chapter_Publication.xlsx']['2023-24'].drop('Download File', axis=1,inplace=True)
dataframes_dict['Book_Chapter_Publication.xlsx']['2022-23'].drop('Download File', axis=1,inplace=True)

KeyError: 'Book_Chapter_Publication.xlsx'

In [31]:
dataframes_dict['Book_Chapter_Publication.xlsx']['2023-24'].columns

KeyError: 'Book_Chapter_Publication.xlsx'

In [32]:
excel_files=list(dataframes_dict.keys())
# Gives list of all the excel files
excel_files

['30 March 2024.xlsx']

In [33]:
def split_authors(df, column_name, max_authors=10):
    if column_name in df.columns:
        df.loc[:,column_name] = df[column_name].str.strip().str.replace(r',+', ',', regex=True).str.rstrip(',')
    else:
        return df

    # Split the 'author' column into lists of authors
    authors_split = df[column_name].str.split(',', expand=True)
    print(authors_split.shape[0])
    print(authors_split.shape[1])
    # Rename columns to 'author_1', 'author_2', ..., 'author_10'
    authors_split.columns = [f'{column_name}{i+1}' for i in range(authors_split.shape[1])]
    
    # If there are less columns than max_authors, add additional columns with None
    for i in range(authors_split.shape[1], max_authors):
        authors_split[f'{column_name}{i+1}'] = None
    
    # # Trim to max_authors columns
    # authors_split = authors_split.iloc[:, :max_authors] # if more than 10 authors are there than it will remove the extra authors
    
    # Drop the original 'author' column and concatenate the new columns
    df = df.drop(columns=[column_name])
    df = pd.concat([df, authors_split], axis=1)
    
    return df
# df = split_authors(dataframes_dict['Book_Chapter_Publication.xlsx']['2022-23'], 'Author', max_authors=10)


In [34]:
for filename, value in dataframes_dict.items():
    # print("File: ",filename)
    sheets=list(value.keys())
    cols=list(value[sheets[0]].columns)
    # print(cols)
    for k,v in value.items():
        # print(v.columns)
        if list(v.columns)!=cols:
            print("Error: Different columns in sheet",k)

In [35]:
list(value[sheets[0]].columns)

['Download File',
 'id',
 'Paper Title',
 'FileName',
 'Name of Journal',
 'JournalType',
 'Impact Factor (Clarivate Analytics)',
 'Impact Factor (Journal)',
 'Year of Publication',
 'Month of Publication',
 'IndexIn',
 'ISSN No',
 'Voume No',
 'Issue No',
 'Page No',
 'Author1',
 'Author2',
 'Author3',
 'Author4',
 'Author5',
 'Author6',
 'Author7',
 'Author8',
 'Author9',
 'Author10',
 'WebsiteJournalLink',
 'ArticleLink',
 'Institute Name',
 'Department Name']

In [36]:
Tables={}
for filename, value in dataframes_dict.items():
    # print("File: ",filename)
    sheets=list(value.keys())
    cols=list(value[sheets[0]].columns)
    # print(cols)
    Dataframe=value[sheets[0]].drop('id',axis=1)
    for i in range(1,len(sheets)):
        Dataframe=pd.concat([Dataframe,value[sheets[i]].drop('id',axis=1)],ignore_index=True).drop_duplicates()
    Tables[filename]=Dataframe
        
        

In [23]:
Tables['Journal_Papers.xlsx']['Unnamed: 24'].unique()

KeyError: 'Journal_Papers.xlsx'

In [37]:
df.head()

AttributeError: 'dict' object has no attribute 'head'

In [38]:
load_dotenv()


True

In [39]:
# Function to create a PostgreSQL database and insert data into tables
def create_db_and_insert_data(db_name, sheet_data):
    # Create a connection to PostgreSQL
    engine = create_engine(f'postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}')
    for sheet_name, df in sheet_data.items():
        print(sheet_name)
        df.to_sql(sheet_name, engine, if_exists='replace', index=False)

In [40]:
for i in excel_files:
    for df in dataframes_dict[i]:
        print(i)
        dataframes_dict[i][df]=split_authors(dataframes_dict[i][df], 'Author', max_authors=10)

30 March 2024.xlsx


In [41]:
for i in excel_files:
    create_db_and_insert_data(i[0:-5].lower(),dataframes_dict[i])

30 March 2024


In [ ]:
# SELECT 'CREATE DATABASE exp_db' 
# WHERE NOT EXISTS (SELECT FROM pg_database WHERE datname = 'exp_db')